# Defining global variables

In [10]:
import json
import os

# import circuit config
circuit = json.load(open('config.json'))
print(circuit)

parallelism = 1 # number of gates that can maximally be executed in parallel

registers_translation = {label:index for index, label in enumerate(circuit["input_registers"])}
reservable_registers = {label:False for label in circuit["input_registers"]} # label of register : is register free to use

num_registers = len(registers_translation)
num_free_registers = 0

imply_logic_strings_parallel = []
AND_count = 0
OR_count = 0
NOT_count = 0
XOR_count = 0

{'input_registers': ['IN_a0', 'IN_a1', 'IN_a2', 'IN_a3', 'IN_a4', 'IN_a5', 'IN_a6', 'IN_a7', 'IN_b0', 'IN_b1', 'IN_b2', 'IN_b3', 'IN_b4', 'IN_b5', 'IN_b6', 'IN_b7', 'IN0', 'IN1', 'IN2', 'IN3', 'IN4', 'IN5', 'IN6', 'IN7', 'IN8', 'IN9', 'IN10', 'IN11', 'IN12', 'IN13'], 'stages': [{'gates': [{'type': 'AND', 'name': 'AND0', 'inputs': ['IN_a7', 'IN_b7']}], 'free_registers_after_stage': []}, {'gates': [{'type': 'AND', 'name': 'AND1', 'inputs': ['IN_a7', 'IN_b6']}], 'free_registers_after_stage': []}, {'gates': [{'type': 'AND', 'name': 'AND2', 'inputs': ['IN_a6', 'IN_b7']}], 'free_registers_after_stage': []}, {'gates': [{'type': 'XOR', 'name': 'XOR0', 'inputs': ['AND1', 'AND2']}], 'free_registers_after_stage': []}, {'gates': [{'type': 'AND', 'name': 'AND3', 'inputs': ['AND1', 'AND2']}], 'free_registers_after_stage': ['AND1', 'AND2']}, {'gates': [{'type': 'AND', 'name': 'AND4', 'inputs': ['IN_a5', 'IN_b7']}], 'free_registers_after_stage': []}, {'gates': [{'type': 'AND', 'name': 'AND5', 'inputs'

# Helper functions to administer registers

In [11]:
def allocate_registers(num_alloc):
    global num_registers, num_free_registers, reservable_registers
    # allocate new registers if needed
    startRegisterLabel = len(registers_translation)-len(circuit["input_registers"])
    for i in range(num_alloc):
        new_register_label = f"w{startRegisterLabel+i}"
        new_register_idx = num_registers

        registers_translation.update({new_register_label:new_register_idx})
        reservable_registers.update({new_register_label:True})

        num_registers = num_registers + 1
        num_free_registers = num_free_registers + 1

def reserve_registers(num_req_registers):
    global num_registers, num_free_registers, reservable_registers
    # check if enough free registers are available
    if num_req_registers > num_free_registers:
        allocate_registers(num_req_registers - num_free_registers)

    reserved_registers = []
    for _ in range(num_req_registers):
        # look for a free register
        for register, status in reservable_registers.items():
            if status == False:
                continue
            
            reserved_registers.append(register)
            reservable_registers[register] = False
            num_free_registers = num_free_registers - 1
            break

    return reserved_registers

def free_registers(registers):
    global reservable_registers, num_free_registers
    for register in registers:
        reservable_registers[register] = True
        num_free_registers = num_free_registers + 1

def rename_register(old, new):
    global registers_translation, reservable_registers

    tmp = registers_translation[old]
    del registers_translation[old]
    registers_translation.update({new:tmp})

    tmp = reservable_registers[old]
    del reservable_registers[old]
    reservable_registers.update({new:tmp})

# Defining Gates

In [12]:
def OR(inputs):
    '''
        creates imply logic for implementing a OR gate

        expects:
            inputs: labels of both input registers

        returns:
            result: label of register in which result is saved
            imply_logic_strings: resulting imply logic in format usable by the ATOMIC tool
    '''
    global OR_count
    OR_count = OR_count + 1
    inp1, inp2 = inputs
    imply_logic_strings = []
    work1, work2 = reserve_registers(2)
    imply_logic_strings.append(f"F{registers_translation[work1]},{registers_translation[work2]}    # OR{OR_count}")
    imply_logic_strings.append(f"I{registers_translation[inp1]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work2]}")
    imply_logic_strings.append(f"F{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[inp2]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work2]}    # ENDE OR{OR_count}")
    result = work2
    to_be_freed = [work1]

    return result, imply_logic_strings, to_be_freed


def AND(inputs):
    '''
        creates imply logic for implementing a AND gate

        expects:
            inputs: labels of both input registers

        returns:
            result: label of register in which result is saved
            imply_logic_strings: resulting imply logic in format usable by the ATOMIC tool
    '''
    global AND_count
    AND_count = AND_count + 1
    inp1, inp2 = inputs
    imply_logic_strings = []
    work1, work2 = reserve_registers(2)
    imply_logic_strings.append(f"F{registers_translation[work1]},{registers_translation[work2]}    # AND{AND_count}")
    imply_logic_strings.append(f"I{registers_translation[inp1]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[inp2]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work2]}    # ENDE AND{AND_count}")
    result = work2
    to_be_freed = [work1]

    return result, imply_logic_strings, to_be_freed

def XOR(inputs):
    global XOR_count
    XOR_count = XOR_count + 1
    inp1, inp2 = inputs
    imply_logic_strings = []
    work1, work2, work3 = reserve_registers(3)

    imply_logic_strings.append(f"F{registers_translation[work1]},{registers_translation[work2]}    # XOR{XOR_count}")
    imply_logic_strings.append(f"F{registers_translation[work1]},{registers_translation[work2]}")
    imply_logic_strings.append(f"I{registers_translation[inp1]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[inp2]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work2]}")
    imply_logic_strings.append(f"F{registers_translation[work1]},{registers_translation[work3]}")
    imply_logic_strings.append(f"I{registers_translation[inp1]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work3]}")
    imply_logic_strings.append(f"F{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[inp2]},{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work1]},{registers_translation[work3]}")
    imply_logic_strings.append(f"I{registers_translation[work3]},{registers_translation[work2]}")
    imply_logic_strings.append(f"F{registers_translation[work1]}")
    imply_logic_strings.append(f"I{registers_translation[work2]},{registers_translation[work1]}     # ENDE XOR{XOR_count}")

    result = work1
    to_be_freed = [work2, work3]
    return result, imply_logic_strings, to_be_freed

def NOT(input):
    '''
        creates imply logic for implementing a NOT gate

        expects:
            inputs: label of input register

        returns:
            result: label of register in which result is saved
            imply_logic_strings: resulting imply logic in format usable by the ATOMIC tool
    '''
    global NOT_count
    NOT_count = NOT_count + 1
    inp = input[0]
    imply_logic_strings = []
    work = reserve_registers(1)[0]
    # print(f"work: {work}")
    imply_logic_strings.append(f"F{registers_translation[work]}    # NOT{NOT_count}")
    imply_logic_strings.append(f"I{registers_translation[inp]},{registers_translation[work]}    # ENDE NOT{NOT_count}")
    result = work
    to_be_freed = []
    return result, imply_logic_strings, to_be_freed

def OUT(input):
    return input[0], [], []
    

gate_mapping = {"OR": OR, "AND": AND, "NOT": NOT, "XOR": XOR, "OUT": OUT}

# Circuit flow functions

In [13]:
def pad_list(lst, length):
    while len(lst) < length:
        lst.append("nop")
    return lst

In [14]:
def combine_logic_chunks(chunk):
    '''
        combines operations that can be run in parallel
    '''

    # print(chunk[0])
    # print(chunk[1])

    while len(chunk) < parallelism:
        chunk.append([])

    # return None
    # if only one gate is present - only unpack and return
    if len(chunk) == 1:
        # print("only one gate present")
        return chunk[0]

    output = []

    # pad gatelogics to length of largest (first) gate
    length = len(chunk[0])
    for i in range(len(chunk[1:])):
        chunk[i+1] = pad_list(chunk[i+1], length)

    # print(chunk[0])
    # print(chunk[1])

    for parallel_logics in zip(*chunk):
        output.append(" | ".join(parallel_logics))

    # print("output")
    # print(output)

    return output



def process_stage(stage):
    current_stage = circuit["stages"][stage]
    # print(f"Stage {stage}: {current_stage}")
    generated_imply_logics = []
    maxSteps = 0

    to_be_freed = []

    # process gates in current stage
    for gate in current_stage["gates"]:
        func = gate_mapping[gate["type"]]
        outputName = gate['name']
        inputs = gate["inputs"]

        out, imply_logic, free_regs = func(inputs)
        generated_imply_logics.append(imply_logic)
        to_be_freed.extend(free_regs)
        if len(imply_logic) > maxSteps:
            maxSteps = len(imply_logic)

        rename_register(out, outputName)
    
    free_registers(to_be_freed)

    generated_imply_logics = sorted(generated_imply_logics, key=lambda x: len(x), reverse=True)
    
    # print("generated_imply_logics")
    # print(generated_imply_logics)

    # combine into chunks that will be processed in parallel
    generated_imply_logics = [generated_imply_logics[i: i+parallelism] for i in range(0, len(generated_imply_logics), parallelism)]
    # print(generated_imply_logics)
    # combine chunks into parallelized imply logic statements
    imply_logic = []
    for chunk in generated_imply_logics:
        # combine_logic_chunks(chunk)
        imply_logic.extend(combine_logic_chunks(chunk))

    # print(f"cleaning up registers {current_stage['free_registers_after_stage']}")
    free_registers(current_stage['free_registers_after_stage'])
    
    return imply_logic

In [26]:
circuit = json.load(open('config.json'))
parallelism = 1 # number of gates that can maximally be executed in parallel

registers_translation = {label:index for index, label in enumerate(circuit["input_registers"])}
reservable_registers = {label:False for label in circuit["input_registers"]} # label of register : is register free to use

num_registers = len(registers_translation)
num_free_registers = 0

imply_logic_strings_parallel = []

logic_result = []
AND_count = -1
OR_count = -1
NOT_count = -1
XOR_count = -1

# logic_result.extend(process_stage(0))

for stage in range(len(circuit["stages"])):
    logic_result.extend(process_stage(stage))
    # print(f"Free Memristors {num_free_registers}/{len(reservable_registers)}")
    # print(len(reservable_registers))
    # print(num_free_registers)   

with open('atomic_config.txt', 'w') as f:
    f.write('\n'.join(logic_result))

print(f"#Memristors: {len(reservable_registers)}")
print(f"Outputs: ")
for out in [rist for rist,free in reservable_registers.items() if not free and "OUT" in rist]:
    print(f"\tLabel: {out} | Index: {registers_translation[out]}")
# print('\n'.join(logic_result))
# print('\n'.join(process_stage(0)))


#Memristors: 62
Outputs: 


In [27]:
# output_labels = ['OR10', 'XOR22', 'XOR20', 'XOR16', 'XOR10', 'XOR4', 'XOR0', 'AND0']
# output_labels = ['XOR128', 'XOR127', 'XOR126', 'XOR125', 'XOR123', 'XOR119', 'XOR115', 'XOR111', 'XOR107', 'XOR103', 'XOR99', 'XOR95', 'XOR10', 'XOR4', 'XOR0', 'AND0']
# output_labels = ['OR9', 'AND36', 'XOR20', 'XOR16', 'XOR10', 'XOR4', 'XOR0', 'AND0']
# output_labels = ['OR165', 'OR158', 'OR151', 'OR144', 'OR137', 'OR130', 'OR123', 'OR116', 'OR109', 'OR102', 'OR95', 'OR88', 'OR81', 'OR74', 'OR67', 'XOR129']
# output_labels = ['OR109', 'OR102', 'OR95', 'OR88', 'OR81', 'OR74', 'OR67', 'XOR137', 'XOR136', 'XOR135', 'XOR134', 'XOR133', 'XOR132', 'XOR131', 'XOR130', 'XOR129']
# output_labels = ['OR176', 'OR169', 'OR162', 'OR155', 'OR148', 'OR141', 'OR134', 'OR127', 'OR120', 'OR113', 'OR106', 'OR99', 'OR92', 'OR85', 'OR78', 'XOR71']
output_labels = ['OR120', 'OR113', 'OR106', 'OR99', 'OR92', 'OR85', 'OR78', 'XOR79', 'XOR78', 'XOR77', 'XOR76', 'XOR75', 'XOR74', 'XOR73', 'XOR72', 'XOR71']

In [55]:
print(reservable_registers)

{'IN_a0': False, 'IN_a1': False, 'IN_a2': False, 'IN_a3': False, 'IN_b0': False, 'IN_b1': False, 'IN_b2': False, 'IN_b3': False, 'IN0': False, 'IN1': False, 'IN2': False, 'w0': True, 'AND0': False}


In [28]:
print("Free memristors:")
print([f"{rist}: {registers_translation[rist]}" for rist,free in reservable_registers.items() if rist in output_labels])

Free memristors:
['XOR71: 11', 'XOR72: 31', 'XOR73: 30', 'XOR74: 29', 'XOR75: 28', 'XOR76: 27', 'XOR77: 26', 'XOR78: 25', 'XOR79: 24', 'OR78: 4', 'OR85: 13', 'OR92: 50', 'OR99: 39', 'OR106: 23', 'OR113: 36', 'OR120: 0']


In [28]:
print(reservable_registers)
print(num_free_registers)

{'IN63': True, 'XOR123': True, 'XOR125': True, 'XOR127': True, 'XOR128': True, 'NOT40': True, 'OR84': True, 'NOT37': True, 'NOT41': True, 'NOT43': True, 'OR85': True, 'NOT35': True, 'OR87': True, 'OR90': True, 'OR86': True, 'NOT38': True, 'NOT39': True, 'NOT42': True, 'NOT36': True, 'NOT44': True, 'XOR134': True, 'OR91': True, 'NOT47': True, 'OR96': True, 'NOT50': True, 'OR92': True, 'NOT45': True, 'OR94': True, 'NOT51': True, 'OR93': True, 'NOT48': True, 'NOT49': True, 'OR97': True, 'NOT46': True, 'NOT52': True, 'NOT53': True, 'OR98': True, 'NOT54': True, 'XOR135': True, 'NOT57': True, 'OR99': True, 'NOT55': True, 'OR101': True, 'OR103': True, 'OR100': True, 'NOT58': True, 'NOT59': True, 'NOT60': True, 'NOT56': True, 'NOT61': True, 'NOT63': True, 'OR104': True, 'NOT64': True, 'XOR136': True, 'NOT67': True, 'NOT62': True, 'OR108': True, 'OR105': True, 'NOT69': True, 'OR106': True, 'NOT65': True, 'OR107': True, 'NOT68': True, 'NOT66': True, 'OUT0': False, 'OUT1': False, 'OUT2': False, '

In [5]:
print(process_stage(0))

cleaning up registers ['i1', 'i2', 'i3', 'i4']
['F4,5 | F6,7', 'I0,4 | I2,6', 'I4,5 | I6,7', 'F4 | F6', 'I1,4 | I3,6', 'I4,5 | I6,7', 'F8,9 | F10,11', 'I0,8 | I2,10', 'I8,9 | I10,11', 'F8 | F10', 'I1,8 | I3,10', 'I8,9 | I10,11']


In [7]:
print(process_stage(1))

only one gate present
cleaning up registers ['or1', 'or2']
['F0,1', 'I5,0', 'I7,0', 'I0,1']


In [8]:
reservable_registers

{'i1': True,
 'i3': True,
 'i4': True,
 'w0': True,
 'or1': True,
 'w2': True,
 'or2': True,
 'w4': True,
 'or3': False,
 'w6': True,
 'or4': False,
 'and1': False}

In [5]:
chunkSize = 2
sortedList = sorted(["abc", "abcde", "ab"], key=lambda x: len(x), reverse=True)
print(sortedList)
chunked = [sortedList[i:i+chunkSize] for i in range(0, len(sortedList), chunkSize)]
print(chunked)

['abcde', 'abc', 'ab']
[['abcde', 'abc'], ['ab']]


In [16]:
process_stage(0)

{'type': 'OR', 'name': 'or1', 'inputs': ['i1', 'i2']}
{'type': 'OR', 'name': 'or2', 'inputs': ['i3', 'i4']}
cleaning up registers ['i1', 'i2', 'i3', 'i4']


In [3]:
my_reg = reserve_registers(2)

In [4]:
print(reservable_registers)
print(num_free_registers)
print(registers_translation)
print(num_registers)
print(my_reg)


{'i1': False, 'i2': False, 'i3': False, 'i4': False, 'w0': False, 'w1': False}
0
{'i1': 0, 'i2': 1, 'i3': 2, 'i4': 3, 'w0': 4, 'w1': 5}
6
['w0', 'w1']


### Test OR Logic

In [9]:
print(reservable_registers)
print(num_free_registers)
print(registers_translation)
print(num_registers)

result, logic = OR("i1", "i2")

print(result)
print(logic)

{'i1': False, 'i2': False, 'i3': False, 'i4': False, 'w0': True, 'w1': False, 'w2': False}
1
{'i1': 0, 'i2': 1, 'i3': 2, 'i4': 3, 'w0': 4, 'w1': 5, 'w2': 6}
7
{'i1': False, 'i2': False, 'i3': False, 'i4': False, 'w0': False, 'w1': False, 'w2': False}
i1
['F4', 'I1,4', 'I4,0']


### Test AND Logic

In [5]:
print(reservable_registers)
print(num_free_registers)
print(registers_translation)
print(num_registers)

result, logic = AND("i1", "i2")

print(result)
print(logic)

{'i1': False, 'i2': False, 'i3': False, 'i4': False}
0
{'i1': 0, 'i2': 1, 'i3': 2, 'i4': 3}
4
w1
['F4,5', 'I0,4', 'I1,4', 'I4,5']


In [7]:
print(reservable_registers)
print(num_free_registers)
print(registers_translation)
print(num_registers)

{'i1': False, 'i2': False, 'i3': False, 'i4': False, 'w0': True, 'w1': False}
1
{'i1': 0, 'i2': 1, 'i3': 2, 'i4': 3, 'w0': 4, 'w1': 5}
6
